Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [124]:
# linear logistic regression with regularization

train_subset = 5000
graph = tf.Graph()
with graph.as_default():
    X = tf.constant(train_dataset[:train_subset, :])
    Y = tf.constant(train_labels[:train_subset])
    W = tf.Variable(tf.zeros((image_size ** 2, num_labels)))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    beta = 0.005
    regularization =  beta * tf.nn.l2_loss(W)
    logits = tf.matmul(X, W) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, Y)) + regularization
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_dataset[:train_subset, :], W) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_dataset[:train_subset, :], W) + biases)
    

In [125]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 2.302572
Training accuracy: 9.9%
Validation accuracy: 62.8%
Loss at step 100: 0.616822
Training accuracy: 85.2%
Validation accuracy: 80.1%
Loss at step 200: 0.589668
Training accuracy: 86.3%
Validation accuracy: 79.9%
Loss at step 300: 0.579011
Training accuracy: 87.0%
Validation accuracy: 80.1%
Loss at step 400: 0.574008
Training accuracy: 87.4%
Validation accuracy: 80.1%
Loss at step 500: 0.571448
Training accuracy: 87.7%
Validation accuracy: 79.9%
Loss at step 600: 0.570059
Training accuracy: 87.9%
Validation accuracy: 79.8%
Loss at step 700: 0.569269
Training accuracy: 88.0%
Validation accuracy: 79.7%
Loss at step 800: 0.568799
Training accuracy: 88.1%
Validation accuracy: 79.6%
Test accuracy: 86.8%


In [170]:
# SGD 1-hiden layer NN with regularization

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  hidden_units = 1024
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_units]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_units, num_labels]))
  biases1 = tf.Variable(tf.zeros([hidden_units]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(layer1, weights2) + biases2
  beta = 0.002
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
    + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [171]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 910.290771
Minibatch accuracy: 5.5%
Validation accuracy: 22.7%
Minibatch loss at step 500: 228.726425
Minibatch accuracy: 90.6%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 83.927765
Minibatch accuracy: 98.4%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 31.004286
Minibatch accuracy: 98.4%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 11.676530
Minibatch accuracy: 97.7%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 4.573780
Minibatch accuracy: 98.4%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 2.043971
Minibatch accuracy: 96.9%
Validation accuracy: 85.7%
Test accuracy: 93.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [187]:
num_steps = 3001

small_train_dataset = train_dataset[:256]
small_train_labels = train_labels[:256]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 962.892212
Minibatch accuracy: 6.2%
Validation accuracy: 33.0%
Minibatch loss at step 500: 231.148529
Minibatch accuracy: 100.0%
Validation accuracy: 66.4%
Minibatch loss at step 1000: 84.992653
Minibatch accuracy: 100.0%
Validation accuracy: 66.2%
Minibatch loss at step 1500: 31.251534
Minibatch accuracy: 100.0%
Validation accuracy: 66.5%
Minibatch loss at step 2000: 11.492323
Minibatch accuracy: 100.0%
Validation accuracy: 66.9%
Minibatch loss at step 2500: 4.236463
Minibatch accuracy: 100.0%
Validation accuracy: 69.0%
Minibatch loss at step 3000: 1.584672
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Test accuracy: 75.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [148]:
# SGD 1-hidden layer NN with drop

batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  hidden_units = 1024
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_units]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_units, num_labels]))
  biases1 = tf.Variable(tf.zeros([hidden_units]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  # Add dropout to layer1, the hidden layer
  keep_ratio = tf.placeholder('float')
  layer1_dropout = tf.nn.dropout(layer1, keep_ratio)
  logits = tf.matmul(layer1_dropout, weights2) + biases2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), keep_ratio), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), keep_ratio), weights2) + biases2)

In [151]:
num_steps = 3001

small_train_dataset = train_dataset[:256]
small_train_labels = train_labels[:256]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_ratio: 0.25}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_ratio: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_ratio: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 733.598633
Minibatch accuracy: 9.4%
Validation accuracy: 34.9%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.5%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.7%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.5%
Minibatch loss at step 2500: 0.004778
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.4%
Test accuracy: 77.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [11]:
# SGD n-hidden layer NN with drop

batch_size = 512

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Hideen unit sizes
  layer1_size = 1024
  layer2_size = 512
  layer3_size = 256
  #layer4_size = 1024

  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(5e-2, global_step, 300, 0.96)
    
  # Weights.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer1_size], stddev=np.sqrt(1.0 / (image_size * image_size))))
  weights2 = tf.Variable(tf.truncated_normal([layer1_size, layer2_size], stddev=np.sqrt(1.0/layer1_size)))
  weights3 = tf.Variable(tf.truncated_normal([layer2_size, layer3_size], stddev=np.sqrt(1.0/layer2_size)))
  #weights4 = tf.Variable(tf.truncated_normal([layer3_size, layer4_size], stddev=np.sqrt(1.0/layer3_size)))
  weights_o = tf.Variable(tf.truncated_normal([layer3_size, num_labels], stddev=np.sqrt(1.0/layer3_size)))

  # Biases
  biases1 = tf.Variable(tf.zeros([layer1_size]))
  biases2 = tf.Variable(tf.zeros([layer2_size]))
  biases3 = tf.Variable(tf.zeros([layer3_size]))
  #biases4 = tf.Variable(tf.zeros([layer4_size]))
  biases_o = tf.Variable(tf.zeros([num_labels]))
  
  # drop keep prob.
  keep_prob = tf.placeholder('float')

  # Training computation.
  layer1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), keep_prob)
  layer2 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer1, weights2) + biases2), keep_prob)
  layer3 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer2, weights3) + biases3), keep_prob)
  #layer4 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer3, weights4) + biases4), keep_prob)
  
  logits = tf.matmul(layer3, weights_o) + biases_o
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

#   beta = 1e-2
#   regularization = beta * (
#     tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights_o))
#   loss += regularization

  # Optimizer.
  optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)
    
  # validation
  valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_h2 = tf.nn.relu(tf.matmul(valid_h1, weights2) + biases2)
  valid_h3 = tf.nn.relu(tf.matmul(valid_h2, weights3) + biases3)
  #valid_h4 = tf.nn.relu(tf.matmul(valid_h3, weights4) + biases4)
  valid_logits = tf.matmul(valid_h3, weights_o) + biases_o
  valid_prediction = tf.nn.softmax(valid_logits)

  # test
  test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_h2 = tf.nn.relu(tf.matmul(test_h1, weights2) + biases2)
  test_h3 = tf.nn.relu(tf.matmul(test_h2, weights3) + biases3)
  #test_h4 = tf.nn.relu(tf.matmul(test_h3, weights4) + biases4)
  test_logits = tf.matmul(test_h3, weights_o) + biases_o
  test_prediction = tf.nn.softmax(test_logits)

In [12]:
num_steps = 60001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: .50}
    _, l, predictions = session.run([optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
#       print("Learning rate: {}".format(learning_rate.eval()))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.333622
Minibatch accuracy: 9.8%
Validation accuracy: 11.9%
Minibatch loss at step 500: 0.616037
Minibatch accuracy: 82.4%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 0.423039
Minibatch accuracy: 87.9%
Validation accuracy: 85.2%
Minibatch loss at step 1500: 0.390234
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 2000: 0.312371
Minibatch accuracy: 90.0%
Validation accuracy: 86.3%
Minibatch loss at step 2500: 0.219643
Minibatch accuracy: 93.9%
Validation accuracy: 86.5%
Minibatch loss at step 3000: 0.138169
Minibatch accuracy: 95.5%
Validation accuracy: 86.7%
Minibatch loss at step 3500: 0.130531
Minibatch accuracy: 96.3%
Validation accuracy: 86.4%
Minibatch loss at step 4000: 0.137802
Minibatch accuracy: 95.1%
Validation accuracy: 86.6%
Minibatch loss at step 4500: 0.102994
Minibatch accuracy: 96.9%
Validation accuracy: 86.8%
Minibatch loss at step 5000: 0.087929
Minibatch accuracy: 97.9%
Validation accuracy